In [22]:
#import library
import pandas as pd
import numpy as np
import librosa
import os
from pydub import AudioSegment
from PIL import Image
from scipy.stats import pearsonr, kendalltau
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [37]:
# Function to load tabular data
def load_tabular_data():
    data_path = input("Please enter the path to your tabular data file (CSV, Excel, etc.): ")
    try:
        tabular_data = pd.read_csv(data_path)  # Change this line based on the file format
        return tabular_data 
    except Exception as e:
        print(f"Error loading tabular data: {e}")
        return None
def reduce_missing_values(tabular_data, threshold = 0.5):
    return tabular_data.dropna(thresh=len(tabular_data) * threshold, axis = 1)


# Function to load image data
def load_image_data():
    folder_path = input("Please enter the path to your image folder: ")
    
    try:
        # Get a list of all files in the folder
        image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.gif'))]

        # Iterate through each image file and display it
        for image_file in image_files:
            image_path = os.path.join(folder_path, image_file)
            image = Image.open(image_path)
            return image.show()
    except Exception as e:
        print(f"Error opening images: {e}")
    

# Function to load audio samples
def load_audio_data():
    audio_path = input("Please enter the path to your audio file: ")
    try:
        audio = AudioSegment.from_file(audio_path)
        return audio.raw_data, audio.frame_rate
    except Exception as e:
        print(f"Error loading audio data: {e}")
        return None

# Function to handle dataset loading based on user input
def load_data():
    print("Choose the type of dataset you want to clean:")
    print("1. Tabular Data")
    print("2. Image Samples (Placeholder)")
    print("3. Audio Samples")

    choice = input("Enter the number corresponding to your dataset type: ")

    if choice == "1":
        return load_tabular_data()
    elif choice == "2":
        return load_image_data()
    elif choice == "3":
        return load_audio_data()
    else:
        print("Invalid choice. Please choose a valid dataset type (1, 2, or 3).")
        return None

# Automated Data Cleaning Script
#Tabular Dataset 
    #Reduce the number with large amount of missing value 
    #Reduce the number of columns by using dropping highly correlated column 
    #Interpolation using polynomial technique
    #Using Linear Regression 
    # To find which column need to be splitted
    # Converting to non string value to float
    #PCA
#Image Sample
    #Average blur
    #Gaussian blur
    #Median blur
    #Bilateral blur
    #Salt Pepper noise removable 
    #Missing Region generation 
    #Read in image and convert to grayscale
    #Meshgrid of pixel coordinate 
    # Create a figure of nrows x ncols subplots, and orient it appropriately
    # for the aspect ratio of the image.
    # Convert an integer i to coordinates in the ax array
    # Sample 100, 1,000, 10,000 and 100,000 points and plot the interpolated
    # images in the figure
    #Night vision (enhancing dark images)
#Audio Sample


# Welcome message
def greet_customer():
    print("Welcome to DataCleaner Pro!")
    print("Your trusted partner for automated data cleaning services.")
    print("We ensure your data is pristine, accurate, and ready for analysis.")
    print("Let us handle the cleaning, so you can focus on insights.")
    print("\n")

# Call the greeting function
greet_customer()

# Load data
loaded_data = load_data()
cleaned_data = reduce_missing_values(tabular_data)
cleaned_data
# Rest of your data cleaning script goes here...


Welcome to DataCleaner Pro!
Your trusted partner for automated data cleaning services.
We ensure your data is pristine, accurate, and ready for analysis.
Let us handle the cleaning, so you can focus on insights.


Choose the type of dataset you want to clean:
1. Tabular Data
2. Image Samples (Placeholder)
3. Audio Samples


NameError: name 'tabular_data' is not defined

In [71]:
from scipy.stats import kendalltau
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold, RFE
from sklearn.impute import KNNImputer
from scipy.stats import kendalltau
#import pandas_profiling
import sweetviz
import great_expectations as ge
from fancyimpute import KNN
from PIL import Image, ImageFilter

In [16]:
# Algorithm 1: Checking correlation between columns (Pearson and Kendall Tau)
def check_correlation(data):
    pearson_correlation = data.corr(method='pearson')
    # Selecting numeric columns for Kendall Tau
    numeric_columns = data.select_dtypes(include=np.number).columns
    kendall_tau_correlation = pd.DataFrame(index=numeric_columns, columns=numeric_columns)
    
    for col1 in numeric_columns:
        for col2 in numeric_columns:
            kendall_tau_correlation.loc[col1, col2], _ = kendalltau(data[col1], data[col2])
    
    return pearson_correlation, kendall_tau_correlation

data = pd.read_csv("tabular_data.csv")
# Test the algorithms
pearson_correlation, kendall_tau_correlation = check_correlation(data)
print("\nPearson Correlation Matrix:")
print(pearson_correlation)
print("\nKendall Tau Correlation Matrix:")
print(kendall_tau_correlation)


Pearson Correlation Matrix:
          Age  Salary
Age     1.000   0.975
Salary  0.975   1.000

Kendall Tau Correlation Matrix:
        Age Salary
Age     1.0    1.0
Salary  1.0    1.0


In [30]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

import numpy as np

def reduce_missing_values(data, threshold=0.5):
    return data.dropna(thresh=len(data) * threshold, axis=1)

def drop_highly_correlated_columns(data, correlation_threshold=0.9):
    corr_matrix = data.corr().abs()
    upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > correlation_threshold)]
    return data.drop(to_drop, axis=1)
def interpolate_missing_values(data, method='polynomial'):
    if method == 'polynomial':
        return data.interpolate(method='polynomial', order=2)
    else:
        return data.interpolate()

def impute_with_linear_regression(data):
    # Find the first column with missing values as the target column
    column_with_missing_value = data.columns[data.isnull().any()]
    if column_with_missing_value.empty:
        print("No column with missing value")
        return data
    if not target_column:
        target_column = column_with_missing_value[0]

    target = data[target_column]
    features = data.drop(columns=[target_column])

    missing_values = data[data[target_column].isnull()]
    if missing_values.empty:
        return data

    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    model = LinearRegression()
    model.fit(X_train, y_train)

    predicted_values = model.predict(missing_values.drop(columns=[target_column]))

    data_copy = data.copy()
    data_copy.loc[data_copy[target_column].isnull(), target_column] = predicted_values

    return data_copy


def perform_pca(data, n_components=2):
    # Separate numeric and categorical columns
    numeric_columns = data.select_dtypes(include=['number']).columns
    categorical_columns = data.select_dtypes(exclude=['number']).columns
    # Convert categorical columns to numeric using label encoding
    label_encoder = LabelEncoder()
    data[categorical_columns] = data[categorical_columns].apply(label_encoder.fit_transform)
    # Apply standard scaling to numeric columns
    scaler = StandardScaler()
    data[numeric_columns] = scaler.fit_transform(data[numeric_columns])
    pca = PCA(n_components=n_components)
    principal_components = pca.fit_transform(data)
    return pd.DataFrame(data=principal_components, columns=[f'Principal_Component_{i+1}' for i in range(n_components)])

def automated_data_cleaning_tabular(input_file_path, output_file_path):
    # Read the input file
    tabular_data = pd.read_csv(input_file_path)

    # Perform data cleaning tasks
    tabular_data_missing_values = reduce_missing_values(tabular_data)
    tabular_data_no_correlation = drop_highly_correlated_columns(tabular_data_missing_values)
    tabular_data_interpolated = interpolate_missing_values(tabular_data_no_correlation)
    tabular_data_imputed = impute_with_linear_regression(tabular_data_interpolated)
    #tabular_data_float = convert_to_float(tabular_data_imputed, ['column1', 'column2'])  # Specify columns to convert
    tabular_data_pca = perform_pca(tabular_data_imputed, n_components=2)

    # Save the cleaned data to a new file
    tabular_data_pca.to_csv(output_file_path, index=False)

# Example usage
input_file_path = 'tabular_data.csv'
output_file_path = 'cleaned_data.csv'
automated_data_cleaning_tabular(input_file_path, output_file_path)


No column with missing value


In [ ]:
# Processing we will provide for our data owner 
    #1. We just working with three dataset type such:
        #Tabular dataset
        #Image Sample
        #Audio Sample
        #In the case of Tabular dataset, what will provide for you in data cleaning:
            #1. 


In [34]:
# Function to handle dataset loading based on user input
def load_data():
    print("Choose the type of dataset you want to clean:")
    print("1. Tabular Data")
    print("2. Image Samples (Placeholder)")
    print("3. Audio Samples")

    choice = input("Enter the number corresponding to your dataset type: ")

    if choice == "1":
        return load_tabular_data()
    elif choice == "2":
        return load_image_data()
    elif choice == "3":
        return load_audio_data()
    else:
        print("Invalid choice. Please choose a valid dataset type (1, 2, or 3).")
        return None
load_data()

Choose the type of dataset you want to clean:
1. Tabular Data
2. Image Samples (Placeholder)
3. Audio Samples
Error loading tabular data: [Errno 2] No such file or directory: ''


In [54]:
# Function to load tabular data
def load_tabular_data():
    data_path = input("Please enter the path to your tabular data file (CSV, Excel, etc.): ")
    try:
        tabular_data = pd.read_csv(data_path)  # Change this line based on the file format
        return reduce_missing_values
    except Exception as e:
        print(f"Error loading tabular data: {e}")
        return None
def reduce_missing_values(tabular_data, threshold = 0.5):
    return tabular_data.dropna(thresh=len(tabular_data) * threshold, axis = 1)

def drop_highly_correlated_columns(data, correlation_threshold=0.9):
    corr_matrix = data.corr().abs()
    upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > correlation_threshold)]
    return data.drop(to_drop, axis=1)
def interpolate_missing_values(data, method='polynomial'):
    if method == 'polynomial':
        return data.interpolate(method='polynomial', order=2)
    else:
        return data.interpolate()
# Function to handle dataset loading based on user input
def load_data():
    print("Choose the type of dataset you want to clean:")
    print("1. Tabular Data")
    print("2. Image Samples (Placeholder)")
    print("3. Audio Samples")

    choice = input("Enter the number corresponding to your dataset type: ")

    if choice == "1":
        return load_tabular_data()
    elif choice == "2":
        return load_image_data()
    elif choice == "3":
        return load_audio_data()
    else:
        print("Invalid choice. Please choose a valid dataset type (1, 2, or 3).")
        return None
def cleaned_tabular(initial_file, cleaned_dataset):
    initial_file = load_data()
    tabular_reduce_missing_value = reduce_missing_values(initial_file)
    tabular_drop_highly_correlated = drop_highly_correlated_columns(reduce_missing_values)
    cleaned_data = tabular_drop_highly_correlated()
cleaned_tabular()


TypeError: cleaned_tabular() missing 2 required positional arguments: 'initial_file' and 'cleaned_dataset'

In [75]:
from PIL import Image, ImageFilter
from PIL import Image, ImageOps
from PIL import Image, ImageFilter
import random


In [85]:
import pandas as pd
import numpy as np

# Function to load tabular data
def load_tabular_data():
    data_path = input("Please enter the path to your tabular data file (CSV, Excel, etc.): ")
    try:
        tabular_data = pd.read_csv(data_path)  # Change this line based on the file format
        return tabular_data
    except Exception as e:
        print(f"Error loading tabular data: {e}")
        return None

def reduce_missing_values(tabular_data, threshold=0.5):
    return tabular_data.dropna(thresh=len(tabular_data) * threshold, axis=1)

def drop_highly_correlated_columns(data, correlation_threshold=0.9):
    corr_matrix = data.corr().abs()
    upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > correlation_threshold)]
    return data.drop(to_drop, axis=1)

def interpolate_missing_values(data, method='polynomial'):
    if method == 'polynomial':
        return data.interpolate(method='polynomial', order=2)
    else:
        return data.interpolate()


def perform_pca(data, n_components=2):
    # Separate numeric and categorical columns
    numeric_columns = data.select_dtypes(include=['number']).columns
    categorical_columns = data.select_dtypes(exclude=['number']).columns
    # Convert categorical columns to numeric using label encoding
    label_encoder = LabelEncoder()
    data[categorical_columns] = data[categorical_columns].apply(label_encoder.fit_transform)
    # Apply standard scaling to numeric columns
    scaler = StandardScaler()
    data[numeric_columns] = scaler.fit_transform(data[numeric_columns])
    pca = PCA(n_components=n_components)
    principal_components = pca.fit_transform(data)
    return pd.DataFrame(data=principal_components, columns=[f'Principal_Component_{i+1}' for i in range(n_components)])

# Function to handle dataset loading based on user input
def load_data():
    print("Choose the type of dataset you want to clean:")
    print("1. Tabular Data")
    print("2. Image Samples (Placeholder)")
    print("3. Audio Samples")

    choice = input("Enter the number corresponding to your dataset type: ")

    if choice == "1":
        return load_tabular_data()
    elif choice == "2":
        return load_image_data()
    elif choice == "3":
        return load_audio_data()
    else:
        print("Invalid choice. Please choose a valid dataset type (1, 2, or 3).")
        return None

def cleaned_tabular():
    initial_file = load_data()
    if initial_file is not None:
        print("\nOriginal Dataset:")
        #initial_file.to_csv("original_dataset.csv", index=False)
        #print(initial_file)

        tabular_data = reduce_missing_values(initial_file)
        tabular_data.to_csv("dataset_after_missing_values.csv", index=False)
        #print("\nDataset after reducing missing values:")
        #print(tabular_data.head(10))

        tabular_data_drop_highly_corr = drop_highly_correlated_columns(tabular_data)
        #print("\nDataset after dropping highly correlated columns:")
        #print(tabular_data_drop_highly_corr.head(10))
        
        tabular_data_perform_pca = perform_pca(tabular_data_drop_highly_corr, n_components=2) 
        tabular_data_perform_pca.to_csv("final_cleaned_dataset.csv", index=False)

        #print("\n Dataset after doing pca")
        #print(tabular_data_perform_pca)
        # Save the cleaned data to a new file
        tabular_data_perform_pca.to_csv("data_after_pca", index=False)
cleaned_tabular()

# Function to load image data
def load_image_data(folder_path):
    data_path = input("Please enter the path to your tabular data file (CSV, Excel, etc.): ")
    try:
        # Get a list of all files in the folder
        image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.gif'))]

        # Iterate through each image file and display it
        for image_file in image_files:
            image_path = os.path.join(folder_path, image_file)
            image = Image.open(image_path)
            image.show()
    except Exception as e:
        print(f"Error opening images: {e}")
def apply_sharpening_smoothing(folder_path):
    try:
        # Iterate over all files in the folder
        for filename in os.listdir(folder_path):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
                # Construct the full path to the image
                image_path = os.path.join(folder_path, filename)
                
                # Open the image
                image = Image.open(image_path)
                
                # Apply sharpening
                sharpened_image = image.filter(ImageFilter.SHARPEN)
                
                # Apply smoothing
                smoothed_image = image.filter(ImageFilter.SMOOTH)
                
                # Display the sharpened and smoothed images (optional)
                sharpened_image.show()
                smoothed_image.show()
    
    except Exception as e:
        print(f"Error applying smoothing: {e}")

def apply_histogram_equalization(folder_path):
    try:
        # Iterate over all files in the folder
        for filename in os.listdir(folder_path):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
                # Construct the full path to the image
                image_path = os.path.join(folder_path, filename)
                
                # Open the image
                image = Image.open(image_path)
                
                # Convert the image to grayscale if it's in color
                if image.mode != 'L':
                    image = ImageOps.grayscale(image)
                
                # Perform histogram equalization
                equalized_image = ImageOps.equalize(image)
                
                # Display the equalized image (optional)
                equalized_image.show()
    except Exception as e:
        print(f"Error applying histogram equalization: {e}")        


def generate_missing_regions(folder_path, missing_region_size=100):
    try:
        # Iterate over all files in the folder
        for filename in os.listdir(folder_path):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
                # Construct the full path to the image
                image_path = os.path.join(folder_path, filename)
                
                # Open the image
                image = Image.open(image_path)
                
                # Get image dimensions
                width, height = image.size
                
                # Generate random coordinates for the missing region
                x = random.randint(0, width - missing_region_size)
                y = random.randint(0, height - missing_region_size)
                
                # Create a solid color (you can replace this with a pattern or another image)
                missing_region_color = (255, 255, 255)  # White color
                
                # Paste the missing region onto the image
                image.paste(missing_region_color, (x, y, x + missing_region_size, y + missing_region_size))
                
                # Save the image with the missing region
                output_path = os.path.join(folder_path, f"missing_{filename}")
                image.save(output_path)
                
                # Display the original and modified images (optional)
                image.show()
                Image.open(output_path).show()
    
    except Exception as e:
        print(f"Error generating missing regions: {e}")     

def apply_average_blur(image_folder):
    try:
        # Iterate over all files in the folder
        for filename in os.listdir(image_folder):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
                # Construct the full path to the image
                image_path = os.path.join(image_folder, filename)
                
                # Open the image
                image = Image.open(image_path)
                
                # Apply Average blur
                average_blur_image = image.filter(ImageFilter.BLUR)
                save_path = os.path.join(image_folder, f"average_blur_{filename}")
                average_blur_image.save(save_path)
                
                # Display the original and modified images (optional)
                image.show()
                average_blur_image.show()
    
    except Exception as e:
        print(f"Error applying Average blur: {e}")
        
def cleaned_image():
    folder_path = load_image_data()
    #generate_missing_regions(folder_path, missing_region_size=100)
    #apply_histogram_equalization()
    apply_average_blur(folder_path)
cleaned_image()


# Function to load audio samples
def load_audio_data():
    audio_path = input("Please enter the path to your audio file: ")
    try:
        audio = AudioSegment.from_file(audio_path)
        return audio.raw_data, audio.frame_rate
    except Exception as e:
        print(f"Error loading audio data: {e}")
        return None 

Choose the type of dataset you want to clean:
1. Tabular Data
2. Image Samples (Placeholder)
3. Audio Samples
Error applying Average blur: expected str, bytes or os.PathLike object, not NoneType
